In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from collections import defaultdict
from nltk.tokenize import word_tokenize 
from collections import Counter
import re


In [2]:
movies = pd.read_csv('Data/cleaned_overview_df.csv')
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9863 entries, 0 to 9862
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   original_title     9863 non-null   object 
 1   id                 9863 non-null   int64  
 2   original_language  9863 non-null   object 
 3   popularity         9863 non-null   float64
 4   release_date       9839 non-null   object 
 5   vote_average       9863 non-null   float64
 6   vote_count         9863 non-null   int64  
 7   genre              9863 non-null   object 
 8   overview           9863 non-null   object 
 9   revenue            9863 non-null   int64  
 10  runtime            9858 non-null   float64
 11  tagline            7077 non-null   object 
 12  summary_length     9863 non-null   int64  
 13  Unnamed: 15        9863 non-null   int64  
 14  Action             9863 non-null   int64  
 15  Adventure          9863 non-null   int64  
 16  Animation          9863 

In [3]:
too_general = ['Animation', 'TV Movie', 'Adventure', 'War', 'Family', 'Thriller', 'Documentary', 'History', 'Crime', 'Mystery', 'Drama'] 
movie_genre = [i.strip('[]').split(",") for i in movies.genre]
for movie in (movie_genre):
    for number, genres in enumerate(movie):
        movie[number] = genres.strip("' '")
movies['genre'] =  movie_genre

In [4]:
new = []
for i, genres in enumerate(movies['genre']):
    for genre in genres[:]:
        if genre in too_general:
            genres.remove(genre)
    new.append(genres)

In [5]:
movies['new genre'] = new

,original_title,id,original_language,popularity,release_date,vote_average,vote_count,genre,overview,revenue,...,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,new genre
0,Venom: Let There Be Carnage,580489,en,5401.308,2021-09-30,6.8,1736,"[Science Fiction, Action]",finding host body investigative reporter alien...,424000000,...,0,0,0,0,1,0,0,0,0,"[Science Fiction, Action]"
1,Eternals,524434,en,3365.535,2021-11-03,7.1,622,"[Action, Science Fiction, Fantasy]",eternals team ancient alien living secret thou...,165000000,...,0,0,0,0,1,0,0,0,0,"[Action, Science Fiction, Fantasy]"
2,Dune,438631,en,2911.423,2021-09-15,8.0,3632,"[Action, Science Fiction]",brilliant gifted young man born great destiny ...,331116356,...,0,0,0,0,1,0,0,0,0,"[Action, Science Fiction]"
3,Army of Thieves,796499,en,2552.437,2021-10-27,6.9,555,[Action],mysterious woman recruit bank teller lead grou...,0,...,0,0,0,0,0,0,1,0,0,[Action]
4,Free Guy,550988,en,1850.470,2021-08-11,7.8,3493,"[Comedy, Action, Science Fiction]",bank teller called realizes background charact...,331096766,...,0,0,0,0,1,0,0,0,0,"[Comedy, Action, Science Fiction]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9858,A Grand Day Out,530,en,9.266,1990-05-18,7.5,594,"[Comedy, Science Fiction]",run cheese provides excellent excuse animated ...,0,...,0,0,0,0,1,0,0,0,0,"[Comedy, Science Fiction]"
9859,El cantante,15934,en,10.417,2006-09-12,7.0,80,[Music],rise fall salsa singer told perspective wife l...,0,...,0,1,0,0,0,0,0,0,0,[Music]
9860,How I Live Now,162215,en,9.520,2013-09-10,6.6,705,[Action],american girl holiday countryside family find ...,0,...,0,0,0,0,0,0,1,1,0,[Action]
9861,Once,5723,en,9.267,2007-03-23,7.4,990,"[Music, Romance]",vacuum repairman moonlight street musician hop...,20710513,...,0,1,0,1,0,0,0,0,0,"[Music, Romance]"


In [6]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from skmultilearn.problem_transform import BinaryRelevance
brelevance = BinaryRelevance(classifier=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True), require_dense=[True, True])


In [7]:
movies_df = movies[~movies['new genre'].isin([[''], []])]

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(movies_df['new genre'])

# transform target variable
y = multilabel_binarizer.transform(movies_df['new genre'])
vectorizer = CountVectorizer()
X= movies_df['overview']
X_train_count = vectorizer.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_train_count,y, test_size = .3, random_state = 42)

In [9]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
binary_rel_clf = BinaryRelevance(MultinomialNB())
binary_rel_clf.fit(X_train, y_train)
y_pred = binary_rel_clf.predict(X_test)
accuracy_score(y_pred, y_test)

0.33421052631578946

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

In [11]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_pred, y_test)

0.3074561403508772

In [12]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average="micro")


0.5985592665356908

In [13]:
multilabel_binarizer.inverse_transform(y_pred)


[('Comedy', 'Romance'),
 ('Action',),
 ('Comedy',),
 (),
 ('Action',),
 ('Action',),
 (),
 ('Horror',),
 (),
 ('Action', 'Horror'),
 ('Action',),
 ('Action',),
 ('Action', 'Comedy'),
 ('Fantasy',),
 (),
 ('Comedy',),
 ('Comedy',),
 ('Science Fiction',),
 ('Horror',),
 ('Action', 'Comedy', 'Fantasy'),
 ('Horror',),
 ('Action', 'Comedy'),
 ('Comedy', 'Romance'),
 ('Action', 'Comedy'),
 ('Comedy', 'Horror'),
 ('Horror', 'Science Fiction'),
 ('Comedy',),
 ('Horror',),
 ('Comedy', 'Romance'),
 ('Comedy', 'Fantasy'),
 ('Comedy', 'Romance'),
 ('Action', 'Comedy'),
 ('Action',),
 ('Horror', 'Science Fiction'),
 ('Action', 'Comedy', 'Science Fiction'),
 ('Horror',),
 ('Action',),
 ('Comedy',),
 ('Fantasy', 'Horror'),
 ('Horror',),
 (),
 ('Comedy',),
 ('Action', 'Comedy'),
 ('Action',),
 ('Action',),
 ('Action',),
 ('Comedy',),
 ('Action', 'Comedy'),
 ('Comedy',),
 ('Comedy',),
 ('Comedy',),
 ('Comedy', 'Romance'),
 ('Comedy',),
 ('Comedy',),
 ('Action',),
 ('Comedy', 'Romance'),
 ('Comedy',),
 

In [14]:
multilabel_binarizer.inverse_transform(y_test)

[('Comedy', 'Romance'),
 ('Action',),
 ('Romance',),
 ('Action',),
 ('Fantasy',),
 ('Action', 'Fantasy', 'Horror'),
 ('Romance',),
 ('Action', 'Science Fiction'),
 ('Horror',),
 ('Action',),
 ('Romance',),
 ('Action', 'Science Fiction'),
 ('Action',),
 ('Comedy', 'Fantasy'),
 ('Comedy', 'Fantasy'),
 ('Action', 'Comedy'),
 ('Comedy', 'Fantasy'),
 ('Action', 'Science Fiction'),
 ('Action',),
 ('Action', 'Comedy', 'Fantasy'),
 ('Horror',),
 ('Action',),
 ('Comedy',),
 ('Action', 'Comedy'),
 ('Horror',),
 ('Horror',),
 ('Music',),
 ('Action', 'Horror'),
 ('Comedy',),
 ('Comedy',),
 ('Comedy', 'Romance'),
 ('Action',),
 ('Action',),
 ('Action', 'Science Fiction'),
 ('Action',),
 ('Horror',),
 ('Action',),
 ('Comedy',),
 ('Comedy', 'Horror'),
 ('Horror',),
 ('Action',),
 ('Comedy',),
 ('Fantasy', 'Science Fiction'),
 ('Action',),
 ('Action', 'Science Fiction'),
 ('Action',),
 ('Comedy',),
 ('Action', 'Comedy'),
 ('Romance',),
 ('Comedy', 'Romance'),
 ('Comedy', 'Romance'),
 ('Comedy',),
 ('C